# Model Comparison and Feature Selection

This notebook provides supervised learning tools to understand associations between the target variable and the predictors. The supporting Python code is in ModelTools.py. Many of the functions have additional arguments that are defaulted. For more details look through the code. Also add any functions you find useful.

# Outline 

<!-- MarkdownTOC autolink=true autoanchor=true bracket=round -->

- [Part One - Univariate Analysis](#part-one---univariate-analysis)
    - [Load Data and Specify Variables](#load-data-and-specify-variables)
    - [One Hot Encode Categorical Variables](#one-hot-encode-categorical-variables)
    - [Linear Univariate Analysis](#linear-univariate-analysis)
    - [Random Forest Univariate Analysis](#random-forest-univariate-analysis)
    - [Gradient Boosting Univariate Analysis](#gradient-boosting-univariate-analysis)
- [Part Two - Model Comparison  --  Done in different workbook.]
- [Part Three - Variable Selection](#part-three---variable-selection)
    - [Random Forest](#random-forest)
    - [Gradient Boosting](#gradient-boosting)

<!-- MarkdownTOC -->

<a name="part-one---univariate-analysis"></a>
# Part One - Univariate Analysis

<a name="load-data-and-specify-variables"></a>
## Load Data and Specify Variables

In this section we will load our dataset, print the index of each feature and then specify which variables are predictors, target, controls (optional), and weight (optional). For variables with null values, null_value_cleanup will create IS_NULL indicator variables. If you prefer another method of dealing with missing values, like imputing values (e.g. mean, median, mode), write a function to do it, put it in EDATools, and call it below. 

In [1]:
import EDATools
import ModelTools
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm

dataset = sm.datasets.get_rdataset("Schooling", "Ecdat").data

EDATools.null_value_cleanup(dataset)

# Print the index of each feature.
for i, col in enumerate(dataset.columns):
    print(i, col)

/home/homer/anaconda4/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


0 smsa66
1 smsa76
2 nearc2
3 nearc4
4 nearc4a
5 nearc4b
6 ed76
7 ed66
8 age76
9 daded
10 nodaded
11 momed
12 nomomed
13 momdad14
14 sinmom14
15 step14
16 south66
17 south76
18 lwage76
19 famed
20 black
21 wage76
22 enroll76
23 kww
24 kww_ISNULL
25 iqscore
26 iqscore_ISNULL
27 mar76
28 libcrd14
29 exp76


Set index of predictors, target, controls, weight.

In [2]:
predictors = np.array([2,3,6,7,8,9,10,11,12,14,15,20,23,25,27,28,29])
numeric_cat_index = np.array([2,3,10,12,14,15,20,27,28])
target = 21

<a name="one-hot-encode-categorical-variables"></a>
## One Hot Encode Categorical Variables

For the purpose of modeling we will one-hot encode each categorical variable. We will also re-name our features so that categorical variables follow the convention VariableName-LevelName. This will allow us to keep track of variable level characteristics, in particular variable importance.

In [3]:
dataset_enc = ModelTools.CreateDummyVars(dataset,predictors,target,numeric_cat_index)

In [4]:
for i, col in enumerate(dataset_enc.columns):
    print(i, col)

0 nearc2-no
1 nearc2-yes
2 nearc4-no
3 nearc4-yes
4 nodaded-no
5 nodaded-yes
6 nomomed-no
7 nomomed-yes
8 sinmom14-no
9 sinmom14-yes
10 step14-no
11 step14-yes
12 black-no
13 black-yes
14 mar76-2
15 mar76-3
16 mar76-4
17 mar76-5
18 mar76-6
19 mar76-Null_Value
20 mar76-yes
21 libcrd14-Null_Value
22 libcrd14-no
23 libcrd14-yes
24 ed76
25 ed66
26 age76
27 daded
28 momed
29 kww
30 iqscore
31 exp76
32 wage76


Split the data into a training and validation set, with the validation set containing p_val_size (defaulted to 0.2) portion of the data.

In [5]:
warnings.filterwarnings('ignore')

predictors_enc = np.arange(32)
target_enc = 32
 
X_train, X_val, Y_train, Y_val = ModelTools.TrainHoldSplit(dataset_enc,predictors_enc,target_enc,p_val_size=0.25)

<a name="random-forest-univariate-analysis"></a>
## Random Forest Univariate Analysis

The idea here is to start with all the predictors and iteratively remove one variable at a time, and calculate the loss in overall fit. A larger loss in fit implies a higher marginal importance of the variable. This gives a slightly different perspective than the linear univariate analysis above. Given a group of correlated and highly predictive variables, they will all come through as predictive in the section above since one variable is being fitted at a time, whereas here removing one of these variables should not affect the overall fit too much since the other correlated variables should pick up the lost signal. Here the focus is more on unique contribution of each variable. 
<br>
To avoid over-fitting, p_cross_validations (defaulted to 5) will be used for cross-validation using p_validation_fraction (defaulted to 0.25) for the test set size. In order to reduce run-time, instead of actually removing variables and re-fitting, we will fit the model on the training set and then randomize the variable we would like to remove in order to break any relationship between it and the target variable. This allows us to fit the model once (for each cross-validation set) and infer for each of our predictors instead of re-fitting it for each of our predictors. 

In [6]:
from ModelToolBox.UnivariatePermutations import *
from Models.skModels import *
warnings.filterwarnings('ignore')

features = ModelTools.FeatureIndexes(dataset, dataset_enc, target_enc)
RF_model = sklearn_Model("Random Forest", 'sklearn.ensemble', 'RandomForestRegressor', 'L1')
UnivariatePermutations(RF_model, features, X_train, Y_train, p_hyperparams = 250)

*****************Features sorted by their score:*****************
ed66: 0.09331
kww: 0.03174
mar76: 0.03049
black: 0.02612
iqscore: 0.01461
daded: 0.01167
ed76: 0.01133
momed: 0.00823
libcrd14: 0.00599
nearc2: 0.00505
exp76: 0.00502
sinmom14: 0.00148
age76: 0.00110
nomomed: 0.00078
nearc4: 0.00048
nodaded: -0.00020
step14: -0.00178


<a name="gradient-boosting-univariate-analysis"></a>
## Gradient Boosting Univariate Analysis

This section is analagous to the Random Forest Univariate Analysis section above, except using Gradient Boosting models instead of Random Forest models. 

In [7]:
GB_model = sklearn_Model("Gradient Boosting", 'sklearn.ensemble', 'GradientBoostingRegressor', 'L1')
UnivariatePermutations(GB_model, features, X_train, Y_train, p_hyperparams = 1000)

*****************Features sorted by their score:*****************
ed66: 0.08632
exp76: 0.05026
ed76: 0.04841
iqscore: 0.03948
kww: 0.03591
mar76: 0.02679
black: 0.02012
daded: 0.01535
momed: 0.01295
age76: 0.01225
sinmom14: 0.00875
nearc2: 0.00653
libcrd14: 0.00553
nearc4: 0.00534
step14: 0.00263
nodaded: -0.00076
nomomed: -0.00195


<a name="part-three---variable-selection"></a>
# Part Three - Variable Selection

Once a model has been chosen, we need to decide which predictor variables to include. Variable selection will be done using backward and forward stepwise selection. The backward part of this algorithm begins by starting with all, say $n$, predictor variables and iteratively removing one variable at a time and measuring the change in performance. The worst performing variable is then removed from the model. We repeat the process with the $n-1$ remaining variables, again removing the worst performing variable. This continues until all of the remaining variables make a "significant" contribution to the model fit. The specific stopping criteria will depend on the problem and metric that is chosen. The forward stepwise selection will then be run (if p_forward = True), by iteratively adding each of the variables that were removed in the backward selection. The most significant, if there any, will be added back to the model. This process continues until none of the remaining variables significantly improve the model.
<br>
It may not be immediately clear why the forward iteration is useful. The backward (and forward) stepwise selection is a greedy algorithm and may find a local optimum in the feature space. Running the forward iteration will increase the chance of finding the global optimum, however does not guarantee it. Ideally we would compare the model fit on every subset of our preditor variables, however this is computationally expensive with $O(2^n)$ run-time, compared to $O(n^2)$ run-time with the backward/forward selection.

<a name="random-forest"></a>
## Random Forest

Here we run backward and forward stepwise selection with Random Forest models. Similar to the univariate random forest section we will cross-validate using p_cross_validations splits and p_validation_fraction portion of the data in each test set. And again to reduce run-time, instead of removing variables and re-fitting, we will randomize the variable we would like to remove which should have the same effect. 

In [8]:
from ModelToolBox.ForBack import *

ForBackPremutations(RF_model, features, predictors_enc, X_train, Y_train, X_val, Y_val, p_hyperparams=250, p_regression=True, p_metric='L1 Error')

*******Variable Performance From Final Backward Iteration:*******
ed66: 0.09269
kww: 0.03156
mar76: 0.03135
black: 0.02698
iqscore: 0.01285
ed76: 0.01154
daded: 0.01123
momed: 0.00730
libcrd14: 0.00604
nearc2: 0.00601
exp76: 0.00374
sinmom14: 0.00163
nearc4: 0.00091
age76: 0.00065
nodaded: 0.00042
nomomed: -0.00006
****************The optimal set of variables is:*****************
nearc2
nearc4
ed76
ed66
age76
daded
nodaded
momed
sinmom14
black
kww
iqscore
mar76
libcrd14
exp76
nomomed


<a name="gradient-boosting"></a>
## Gradient Boosting

This section is analagous to the one above, except using gradient boosting.

In [9]:
ForBackPremutations(GB_model, features, predictors_enc, X_train, Y_train, X_val, Y_val, p_hyperparams=1000, p_regression=True, p_metric='L1 Error', p_threshold=0.1)

*******Variable Performance From Final Backward Iteration:*******
ed76: 0.10663
ed66: 0.10343
exp76: 0.09883
kww: 0.09596
****************The optimal set of variables is:*****************
ed76
ed66
exp76
